# 🎨 anime-character-generator v1.5 (LoRA Edition)
## Stable Diffusion v1.5 + LoRA Fine-tuning

**バージョン**: 1.5 (LoRA実装版・課題あり)  
**説明**: Stable Diffusion v1.5 にLoRAによるファインチューニングを適用  
**ガイド**: [Day3-4 ブログ記事](https://shion.blog/)  
**コード**: [character_generator_v1_lora.py](../character_generator_v1_lora.py)

Stable Diffusion v1.5 を **アニメ・印象派風** に特化させるため、LoRA を使用したファインチューニングを Google Colab (T4 GPU) で実行します。

## ⚠️ このノートブックは v1.5 (試行錯誤版) です

このノートブックを実行することで LoRA トレーニングはできますが、以下の**既知の課題**があります。詳細は下記を参照してください。

---

## ⚠️ 既知の課題と v2.0 での解決方法

### 課題 1️⃣: Character-level noise への脆弱性
- **論文**: Gao et al. (2306.13103) 「Text-to-Image Robustness」
- **症状**: 「astronaut」→「astornaut」（1文字違い）で生成結果が劇的に変わる
- **原因**: 単一レイヤーのプロンプト設計で、タイポやグリフ攻撃に対応していない
- **URL**: https://arxiv.org/abs/2306.13103
- **v2.0での解決**: Phase 1 で LLM による多層冗長プロンプト設計を実装

### 課題 2️⃣: 推論速度が遅い
- **実測**: 3.8秒/画像 (T4 GPU, Stable Diffusion v1.5)
- **改善機会**: Latent Consistency Model (LCM) 蒸留で 12x 高速化可能
- **URL**: https://arxiv.org/abs/2310.04378
- **v2.0での解決**: Phase 2B で LCM 蒸留を実装 (1秒/画像 達成予定)

### 課題 3️⃣: マルチモーダル入力未対応
- テキスト入力のみ
- Image-to-Image や ControlNet による スケッチ・ポーズ指定生成 未実装
- **v2.0での解決**: Phase 3 で完全なマルチモーダル対応

### 課題 4️⃣: 本番環境対応がない
- 研究スクリプト形式
- REST API や Web UI がない
- クラウドデプロイ対応なし
- **v2.0での解決**: Phase 4 で Streamlit UI + FastAPI + クラウドデプロイ実装

---

## ✅ これらの課題は v2.0 (Phase 1-4) で段階的に解決されます

詳細: [IMPLEMENTATION_ROADMAP.md](../IMPLEMENTATION_ROADMAP.md)

v2.0 では 以下を段階的に実装します：
- **Phase 1**: Gemini LLM による多層冗長プロンプト設計（Gao et al. 脆弱性対応）
- **Phase 2A**: 改善されたメモリ最適化によるLoRA最適化
- **Phase 2B**: LCM 蒸留による 12x 推論高速化
- **Phase 3**: Image-to-Image + ControlNet + アニメーション生成対応
- **Phase 4**: Streamlit UI + FastAPI バックエンド + クラウドデプロイ

---

## 📚 ブログ記事との対応

このノートブックはブログの Day3-4 記事で説明されたLoRA実装版です。
ブログと完全に整合しており、記事を読みながら実行することで
LoRA ファインチューニングの基礎を学べます。

---

**推奨**: v2.0 の Phase 1-4 実装をお待ちください。

---

## Step 1: 環境確認・GPU セットアップ

In [ ]:
# GPU の確認
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## Step 2: 依存パッケージのインストール

In [ ]:
# 必要なパッケージをインストール（peft不要 - 純粋PyTorch LoRAを使用）
!pip install -q diffusers transformers pillow torch tqdm safetensors

## Step 3: Google Drive にマウント

In [ ]:
from google.colab import drive

# Google Drive をマウント
drive.mount('/content/drive')
print("✅ Google Drive mounted")

## Step 4: データセットの準備

**事前準備 (このノートブック実行前):**
1. ローカルで `python scripts/download_danbooru.py --limit 60` を実行
2. 生成された `training_data/` フォルダを Google Drive にアップロード
3. 以下のセルを実行

In [ ]:
import os
from pathlib import Path

# Google Drive から training_data をコピー
drive_path = Path("/content/drive/MyDrive/training_data")
local_path = Path("/content/training_data")

if drive_path.exists():
    print(f"✅ Found: {drive_path}")
    # シンボリックリンク作成 (速度向上)
    if not local_path.exists():
        os.symlink(drive_path, local_path)
        print(f"✅ Symlink created: {local_path}")
    
    # データセット統計
    import json
    metadata_file = drive_path / "metadata.json"
    if metadata_file.exists():
        with open(metadata_file) as f:
            metadata = json.load(f)
        print(f"📊 Dataset: {len(metadata.get('training_data', []))} images")
else:
    print(f"❌ Not found: {drive_path}")
    print("\nPlease upload training_data/ to Google Drive")

## Step 5: train_lora.py のダウンロード

GitHub から最新の train_lora.py を取得

In [ ]:
import os
import json
import sys
import shutil
import urllib.request
import socket
from pathlib import Path

# ステップ 5: 学習スクリプトをダウンロード&パッチ適用
print("=" * 70)
print("📥 Step 5: Download train_lora.py from GitHub & Apply Device Patches")
print("=" * 70)

# 出力ディレクトリ作成
output_dir = Path("/content/lora_weights")
output_dir.mkdir(parents=True, exist_ok=True)
print(f"✅ Output directory ready: {output_dir}\n")

# データディレクトリを確認
training_data_dir = Path("/content/training_data")
print(f"🔍 Checking training data directory: {training_data_dir}")
print(f"   Exists: {training_data_dir.exists()}")

if training_data_dir.exists():
    print(f"\n📂 Directory structure:")
    style_dirs = []
    for item in sorted(training_data_dir.iterdir()):
        if item.is_dir() and not item.name.startswith("."):
            image_count = len(list(item.glob("*.png"))) + len(list(item.glob("*.jpg"))) + len(list(item.glob("*.jpeg")))
            print(f"   📁 {item.name}/ ({image_count} images)")
            style_dirs.append(item)
        elif item.is_file() and not item.name.startswith("."):
            print(f"   📄 {item.name}")

    total_images = len(list(training_data_dir.rglob("*.png"))) \
                 + len(list(training_data_dir.rglob("*.jpg"))) \
                 + len(list(training_data_dir.rglob("*.jpeg")))
    print(f"\n📊 Total images: {total_images}")

    if total_images == 0:
        print("⚠️  WARNING: No images found!")
        print("   Structure should be: /content/training_data/<style_name>/*.png")
else:
    print("❌ training_data directory not found! Creating...")
    training_data_dir.mkdir(parents=True, exist_ok=True)

# ================== train_lora.py の取得 ==================
print(f"\n📥 Locating train_lora.py...")
train_script = Path("/content/train_lora.py")

github_urls = [
    "https://raw.githubusercontent.com/Shion1124/anime-character-generator/master/train_lora.py",
    "https://raw.githubusercontent.com/Shion1124/anime-character-generator/main/train_lora.py",
]

downloaded = False

def download_with_timeout(url, dest_path, timeout=15):
    """URLからファイルをダウンロード（タイムアウト付き）"""
    socket.setdefaulttimeout(timeout)
    try:
        with urllib.request.urlopen(url) as response:
            content = response.read()
        with open(dest_path, 'wb') as f:
            f.write(content)
    finally:
        socket.setdefaulttimeout(None)

for url in github_urls:
    try:
        print(f"   Trying: {url[:60]}...")
        download_with_timeout(url, train_script)
        print(f"   ✅ Downloaded from GitHub")
        downloaded = True
        break
    except Exception as e:
        print(f"   ⚠️  Failed: {str(e)[:60]}")

if not downloaded:
    local_sources = [Path("./train_lora.py"), Path("../train_lora.py")]
    for local_path in local_sources:
        if local_path.exists():
            shutil.copy(local_path, train_script)
            print(f"   ✅ Copied from {local_path}")
            downloaded = True
            break

if not downloaded:
    raise FileNotFoundError(
        "train_lora.py not found. Run Step 5.5 (Fallback) to get it via git clone."
    )

print(f"\n✅ train_lora.py ready: {train_script.stat().st_size / 1024:.1f} KB")

# ================== パッチ適用 ==================
print(f"\n🔧 Applying patches...")

with open(train_script, "r", encoding="utf-8") as f:
    content = f.read()

patches_applied = 0

# パッチ 1: magnitude device fix
p1_old = 'self.magnitude = nn.Parameter(torch.randn_like(original_weight[:rank]))'
p1_new = 'self.magnitude = nn.Parameter(torch.randn_like(original_weight[:rank]).to(linear.weight.device))'
if p1_old in content:
    content = content.replace(p1_old, p1_new)
    print("  ✅ Patch 1: magnitude device fix")
    patches_applied += 1
else:
    print("  ℹ️  Patch 1: Already applied")

# パッチ 2: use_qlora parameter を train() に追加
if 'def train(' in content and 'use_qlora' not in content:
    # train() シグネチャに use_qlora を追加
    old_sig = 'save_interval: int = 5,\n    ):'
    new_sig = 'save_interval: int = 5,\n        use_qlora: bool = True,\n    ):'
    if old_sig in content:
        content = content.replace(old_sig, new_sig)
        print("  ✅ Patch 2: use_qlora parameter added to train()")
        patches_applied += 1
    else:
        print("  ⚠️  Patch 2: Could not inject use_qlora (manual fix may be needed)")
elif 'use_qlora' in content:
    print("  ℹ️  Patch 2: use_qlora already present")

# パッチ 3: training_log の use_qlora フィールド追加
if '"lora_alpha": self.lora_alpha,' in content and '"use_qlora"' not in content:
    old_log = '"lora_alpha": self.lora_alpha,\n                },'
    new_log = '"lora_alpha": self.lora_alpha,\n                    "use_qlora": use_qlora,\n                },'
    if old_log in content:
        content = content.replace(old_log, new_log)
        print("  ✅ Patch 3: use_qlora added to training_log config")
        patches_applied += 1

with open(train_script, "w", encoding="utf-8") as f:
    f.write(content)

print(f"\n✅ {patches_applied} patches applied")

sys.path.insert(0, "/content")
print(f"✅ Python path updated")

### Step 5.5: Fallback - Create train_lora.py from template (if download fails)

If the download failed, run this cell to create the training script:


In [ ]:
from pathlib import Path
import subprocess
import shutil

print("🔄 Fallback: Creating train_lora.py from git clone...")
train_script = Path("/content/train_lora.py")

if train_script.exists() and train_script.stat().st_size > 10000:
    print(f"✅ train_lora.py already exists ({train_script.stat().st_size / 1024:.1f} KB)")
else:
    print(f"📥 Attempting to clone from GitHub...\n")
    
    success = False
    
    # 方法 1: git clone
    try:
        print("  1️⃣  Trying: git clone")
        result = subprocess.run(
            ["git", "clone", "--depth", "1", 
             "https://github.com/Shion1124/anime-character-generator.git",
             "/tmp/anime-gen"],
            capture_output=True,
            timeout=60,
            text=True
        )
        
        src = Path("/tmp/anime-gen/train_lora.py")
        if src.exists() and src.stat().st_size > 10000:
            shutil.copy(src, train_script)
            print(f"     ✅ Success! Cloned and copied train_lora.py")
            print(f"     📦 Size: {train_script.stat().st_size / 1024:.1f} KB")
            success = True
        else:
            print(f"     ⚠️  Clone succeeded but file not found/too small")
    
    except subprocess.TimeoutExpired:
        print(f"     ⚠️  Timeout (60s)")
    except Exception as e:
        print(f"     ⚠️  Error: {str(e)[:60]}")
    
    # 方法 2: pip install (if git clone failed)
    if not success:
        try:
            print(f"\n  2️⃣  Trying: pip install from repo")
            result = subprocess.run(
                ["pip", "install", "-q", 
                 "git+https://github.com/Shion1124/anime-character-generator.git"],
                capture_output=True,
                timeout=120,
                text=True
            )
            
            # パッケージがインストールされたか確認
            import importlib.util
            spec = importlib.util.find_spec("train_lora")
            if spec is not None:
                print(f"     ✅ Package installed successfully")
                success = True
            else:
                print(f"     ⚠️  Package installed but import failed")
        
        except Exception as e:
            print(f"     ⚠️  Error: {str(e)[:60]}")
    
    # 方法 3: 直接ダウンロード (if both above failed)
    if not success:
        try:
            print(f"\n  3️⃣  Trying: Direct download via urllib")
            import urllib.request
            import socket
            
            urls = [
                "https://raw.githubusercontent.com/Shion1124/anime-character-generator/master/train_lora.py",
                "https://raw.githubusercontent.com/Shion1124/anime-character-generator/main/train_lora.py",
            ]
            
            for url in urls:
                try:
                    socket.setdefaulttimeout(10)
                    with urllib.request.urlopen(url) as response:
                        content = response.read()
                    
                    with open(train_script, 'wb') as f:
                        f.write(content)
                    
                    socket.setdefaulttimeout(None)
                    print(f"     ✅ Downloaded successfully")
                    print(f"     📦 Size: {train_script.stat().st_size / 1024:.1f} KB")
                    success = True
                    break
                except Exception as url_error:
                    socket.setdefaulttimeout(None)
                    print(f"     ⚠️  {url[:50]}... failed")
        
        except Exception as e:
            print(f"     ⚠️  Error: {str(e)[:60]}")

# Final verification
print(f"\n{'='*60}")
if not train_script.exists():
    print(f"❌ FAILED: train_lora.py could not be obtained!")
    print(f"\n🔧 Manual Solution:")
    print(f"   1. Download from: https://github.com/Shion1124/anime-character-generator")
    print(f"   2. Upload train_lora.py to /content/")
    print(f"   3. Run Step 6 again")
    raise FileNotFoundError(
        f"Could not obtain train_lora.py from any source.\n"
        f"Please upload it manually to /content/train_lora.py"
    )
elif train_script.stat().st_size < 10000:
    print(f"⚠️  WARNING: train_lora.py exists but is only {train_script.stat().st_size} bytes!")
    print(f"   It may be incomplete. Consider re-downloading.")
else:
    print(f"✅ SUCCESS: train_lora.py is ready!")
    print(f"   📍 Location: {train_script}")
    print(f"   📦 Size: {train_script.stat().st_size / 1024:.1f} KB")
    print(f"   ✨ You can now run Step 6 (Training)")
print(f"{'='*60}")

## Step 5.6: Emergency Fallback - Embedded train_lora.py

If all other methods fail, this cell can create train_lora.py with embedded code:


In [ ]:
from pathlib import Path

print("🆘 Emergency Fallback: Create train_lora.py\n")

train_script = Path("/content/train_lora.py")

if train_script.exists() and train_script.stat().st_size > 10000:
    print(f"✅ train_lora.py already exists ({train_script.stat().st_size / 1024:.1f} KB)")
    print("   No need to use this cell\n")
else:
    print("📝 Options to get train_lora.py:\n")
    print("Option 1: Upload from GitHub Web UI")
    print("  1. Go to: https://github.com/Shion1124/anime-character-generator/blob/master/train_lora.py")
    print("  2. Click 'Raw' button")
    print("  3. Save the page (Cmd+S)")
    print("  4. Upload to Colab and move to /content/\n")
    
    print("Option 2: Try alternative HTTP method (slow but may work):")
    try:
        import time
        print("  Attempting download...")
        
        # Try with requests library if available
        try:
            import requests
            url = "https://raw.githubusercontent.com/Shion1124/anime-character-generator/master/train_lora.py"
            response = requests.get(url, timeout=30)
            if response.status_code == 200:
                with open(train_script, 'w') as f:
                    f.write(response.text)
                print(f"  ✅ Downloaded successfully using requests!")
                print(f"     Size: {train_script.stat().st_size / 1024:.1f} KB")
            else:
                print(f"  ⚠️  HTTP Error {response.status_code}")
        except:
            # Fallback to urllib
            import urllib.request
            import ssl
            
            # SSL 証明書エラーを無視（開発環境用）
            ssl_context = ssl.create_default_context()
            ssl_context.check_hostname = False
            ssl_context.verify_mode = ssl.CERT_NONE
            
            url = "https://raw.githubusercontent.com/Shion1124/anime-character-generator/master/train_lora.py"
            print(f"  Trying: {url[:60]}...")
            
            request = urllib.request.Request(url)
            request.add_header('User-Agent', 'Mozilla/5.0')
            
            with urllib.request.urlopen(request, context=ssl_context, timeout=30) as response:
                content = response.read().decode('utf-8')
            
            with open(train_script, 'w', encoding='utf-8') as f:
                f.write(content)
            
            print(f"  ✅ Downloaded successfully!")
            print(f"     Size: {train_script.stat().st_size / 1024:.1f} KB")
    
    except Exception as e:
        print(f"  ⚠️  Failed: {str(e)[:80]}\n")
        print("Option 3: Manual creation")
        print("  If you have local access to the file:")
        print("  1. Mount Google Drive: !from google.colab import drive")
        print("  2. Copy from Drive: !cp /content/drive/MyDrive/train_lora.py /content/")
        print("  (Make sure to upload train_lora.py to your Google Drive first)\n")

# Final status
print("\n" + "="*60)
if train_script.exists() and train_script.stat().st_size > 10000:
    print(f"✅ train_lora.py is ready!")
    print(f"   📍 {train_script}")
    print(f"   📦 {train_script.stat().st_size / 1024:.1f} KB")
else:
    print(f"ℹ️  train_lora.py not yet obtained")
    print(f"   Try one of the options above, then run Step 6")
print("="*60)

## Step 6: DoRA + QLoRA トレーニング実行

**DoRA (Dominant Rank Adaptation):** LoRA の進化版  
- **Magnitude**: 低周波スケール成分（画像全体の特性）
- **Direction**: 高周波方向成分（細部・ノイズ・スタイル詳細）

**QLoRA (Quantized LoRA):** 混合精度学習  
- **Direction 成分を float16 量子化** → メモリ削減（～60%）
- **Magnitude** は高精度のまま（学習安定性）
- T4 GPU でも rank=32-64 で訓練可能

**重要:** 
- 標準: 20～30分（10 エポック, rank=32）
- QLoRA: 同時間でメモリ効率 60% 改善
- Colab の接続を保持してください。

In [ ]:
import torch
import sys
import inspect
import traceback
from pathlib import Path

print("=" * 70)
print("🎯 Step 6: DoRA + QLoRA Training")
print("=" * 70)

# 設定パラメータ（T4 OOM 対策版）
EPOCHS = 10
LORA_RANK = 32
BATCH_SIZE = 1  # 2 → 1 (T4 OOM 対策)
GRAD_ACCUM_STEPS = 4  # 勾配蓄積 (実効バッチ = 1 × 4 = 4)
LEARNING_RATE = 1e-5  # 1e-4 → 1e-5 (NaN 対策)
USE_QLORA = False  # True → False (デフォルトで float32、安定性重視)
SAVE_INTERVAL = 5

print(f"""
⚙️  Configuration (T4 Memory-Optimized):
   Epochs: {EPOCHS}
   LoRA Rank: {LORA_RANK}
   Batch Size: {BATCH_SIZE} (reduced for T4 VRAM)
   Gradient Accumulation: {GRAD_ACCUM_STEPS} steps (effective batch = {BATCH_SIZE * GRAD_ACCUM_STEPS})
   Learning Rate: {LEARNING_RATE} (reduced from 1e-4)
   QLoRA: {USE_QLORA} (disabled for numerical stability)
   Mixed Precision: AMP enabled (autocast + GradScaler)
   Gradient Checkpointing: enabled (VRAM 節約)
   Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}

📝 Memory optimizations:
   - AMP (float16 forward, float32 gradients)
   - Gradient checkpointing (recompute activations)
   - Gradient accumulation (batch_size=1 × 4 steps = effective 4)
""")

# VRAM 確認
if torch.cuda.is_available():
    total_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"📊 GPU VRAM: {total_mem:.1f} GB")
    torch.cuda.empty_cache()
    import gc
    gc.collect()
    print("🧹 VRAM cache cleared")

# データセット確認
training_data_dir = Path("/content/training_data")
total_images = len(list(training_data_dir.rglob("*.png"))) \
             + len(list(training_data_dir.rglob("*.jpg"))) \
             + len(list(training_data_dir.rglob("*.jpeg")))

print(f"📊 Dataset check: {total_images} images found")
if total_images == 0:
    print("❌ ERROR: No training images found!")
    sys.exit(1)

# トレーニング実行
try:
    train_script = Path("/content/train_lora.py")
    if not train_script.exists():
        raise FileNotFoundError(f"{train_script} not found! Run Step 5 first.")

    print(f"✅ train_lora.py found: {train_script.stat().st_size / 1024:.1f} KB\n")

    sys.path.insert(0, "/content")

    # キャッシュをクリアして最新版をインポート
    if "train_lora" in sys.modules:
        del sys.modules["train_lora"]

    print("📦 Importing train_lora module...")
    from train_lora import LoRATrainer
    print("✅ Successfully imported LoRATrainer\n")

    print("🚀 Initializing DoRA trainer (AMP mixed precision)...")
    trainer = LoRATrainer(
        model_id="runwayml/stable-diffusion-v1-5",
        output_dir="/content/lora_weights",
        lora_rank=LORA_RANK,
        lora_alpha=LORA_RANK,
        device="cuda" if torch.cuda.is_available() else "cpu"
    )

    # train() が各パラメータをサポートするか動的に確認
    train_sig = inspect.signature(trainer.train)
    train_kwargs = {
        "data_dir": "/content/training_data",
        "num_epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "learning_rate": LEARNING_RATE,
        "save_interval": SAVE_INTERVAL,
    }

    if "use_qlora" in train_sig.parameters:
        train_kwargs["use_qlora"] = USE_QLORA
    if "gradient_accumulation_steps" in train_sig.parameters:
        train_kwargs["gradient_accumulation_steps"] = GRAD_ACCUM_STEPS

    print(f"📝 Starting training ({EPOCHS} epochs × {total_images} images)...")
    print(f"   {'✅ QLoRA enabled (float16)' if USE_QLORA else '✅ Using float32 (numerically stable)'}")
    print(f"   ⚡ AMP + Gradient Checkpointing enabled\n")

    training_log = trainer.train(**train_kwargs)

    print(f"\n✅ Training completed!")

    if training_log:
        history = training_log.get("history", [])
        status = training_log.get("status", "unknown")
        print(f"   Status: {status}")
        print(f"   Epochs completed: {len(history)}")
        if history:
            initial_loss = history[0].get('loss', float('nan'))
            final_loss = history[-1].get('loss', float('nan'))
            print(f"   Initial loss: {initial_loss:.6f}")
            print(f"   Final loss:   {final_loss:.6f}")
            
            # 改善度をチェック
            if initial_loss > 0 and final_loss < initial_loss:
                improvement = (initial_loss - final_loss) / initial_loss * 100
                print(f"   🎯 Improvement: {improvement:.1f}%")
            elif final_loss > initial_loss:
                print(f"   ⚠️  Loss increased - training may be unstable")
            
            # NaN 検出数をチェック
            valid_epochs = sum(1 for h in history if not (h.get('loss', 0) <= 0 or h.get('loss', float('inf')) > 1000))
            print(f"   ✅ Valid epochs: {valid_epochs}/{len(history)}")

except FileNotFoundError as fe:
    print(f"\n❌ File Not Found: {fe}")
    print(f"   Run Step 5 (and Step 5.5 if needed) to obtain train_lora.py")

except ImportError as ie:
    print(f"\n❌ Import Error: {ie}")
    import subprocess
    subprocess.run(
        ["pip", "install", "-q", "diffusers", "transformers", "pillow", "tqdm", "safetensors"],
        timeout=120
    )
    print("✅ Dependencies installed. Please re-run this cell.")
    raise

except Exception as e:
    print(f"\n❌ Training error: {e}")
    traceback.print_exc()

    print(f"\n🔍 Diagnostic:")
    print(f"   Python {sys.version.split()[0]}, PyTorch {torch.__version__}")
    print(f"   CUDA: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"   GPU: {torch.cuda.get_device_name(0)} ({torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB)")
        print(f"   VRAM allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
        print(f"   VRAM reserved: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")

    # エラーログ保存
    import json
    output_dir = Path("/content/lora_weights")
    output_dir.mkdir(parents=True, exist_ok=True)
    error_log = {
        "error": str(e),
        "error_type": type(e).__name__,
        "traceback": traceback.format_exc(),
        "config": {"epochs": EPOCHS, "rank": LORA_RANK, "batch_size": BATCH_SIZE,
                   "gradient_accumulation_steps": GRAD_ACCUM_STEPS,
                   "learning_rate": LEARNING_RATE, "use_qlora": USE_QLORA, "images": total_images}
    }
    with open(output_dir / "error_log.json", "w") as f:
        json.dump(error_log, f, indent=2)
    print(f"   Error saved to: /content/lora_weights/error_log.json")
    raise

print("\n" + "=" * 70)

## Step 7: 学習結果の確認

In [ ]:
import json
from pathlib import Path
import matplotlib.pyplot as plt

print("🔍 Training Result Verification...")
print("="*60)

# 出力ディレクトリ確認
output_dir = Path("/content/lora_weights")
print(f"📁 Output directory: {output_dir}")
print(f"   Exists: {output_dir.exists()}")

if output_dir.exists():
    print(f"\n📂 Contents:")
    for file in sorted(output_dir.glob("*")):
        if file.is_file():
            size = file.stat().st_size
            if size > 1024*1024:
                print(f"   {file.name:40s} {size/(1024*1024):>8.2f} MB")
            else:
                print(f"   {file.name:40s} {size:>8.0f} bytes")

# 学習ログをロード
log_file = Path("/content/lora_weights/training_log.json")
print(f"\n🔎 Looking for training_log.json...")
print(f"   Path: {log_file}")
print(f"   Exists: {log_file.exists()}")

if not log_file.exists():
    # エラーメッセージを診断
    print("\n⚠️  training_log.json not found!")
    print("\n🔴 Possible causes:")
    print("   1. Training failed silently (check AnimeDataset)")
    print("   2. No images in training_data/")
    print("   3. Exception during training (check stderr)")
    print("   4. Output directory not created")
    
    # データセット確認
    data_dir = Path("/content/training_data")
    if data_dir.exists():
        print(f"\n📊 Dataset check:")
        total_images = len(list(data_dir.rglob("*.png"))) + len(list(data_dir.rglob("*.jpg")))
        print(f"   Images found: {total_images}")
        if total_images == 0:
            print("   ❌ No images found! Training cannot proceed.")
    else:
        print(f"\n❌ Data directory not found: {data_dir}")
else:
    # ログファイル読み込み
    try:
        with open(log_file) as f:
            training_log = json.load(f)
        
        print("\n✅ training_log.json loaded successfully")
        
        # 学習が実行されたかチェック
        history = training_log.get("history", [])
        if len(history) == 0:
            print("⚠️  Warning: history is empty (no epochs completed)")
            print("   Config:", json.dumps(training_log.get("config", {}), indent=2))
        else:
            # 学習曲線を表示
            losses = [h['loss'] for h in history]
            epochs = list(range(1, len(losses) + 1))
            
            plt.figure(figsize=(10, 6))
            plt.plot(epochs, losses, 'b-', linewidth=2, marker='o')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title('DoRA Training Loss Curve')
            plt.grid(True, alpha=0.3)
            plt.yscale('log')
            plt.tight_layout()
            plt.savefig('/content/training_curve.png', dpi=100)
            plt.show()
            
            # 統計情報
            print(f"\n✅ Training completed successfully!")
            print(f"📊 Total epochs: {len(losses)}")
            print(f"📉 Final loss: {losses[-1]:.6f}")
            print(f"📈 Initial loss: {losses[0]:.6f}")
            if losses[0] > losses[-1]:
                improvement = (losses[0] - losses[-1]) / losses[0] * 100
                print(f"🎯 Loss improvement: {improvement:.1f}%")
            else:
                print(f"⚠️  Loss increased (may indicate training issues)")
            
            # トレーニング設定表示
            config = training_log.get("config", {})
            print(f"\n⚙️  Training config:")
            for key, value in config.items():
                print(f"   {key}: {value}")
    
    except json.JSONDecodeError as e:
        print(f"\n❌ Error parsing training_log.json: {e}")
    except Exception as e:
        print(f"\n❌ Error: {e}")

print("\n" + "="*60)

## Step 6.5: Diagnostic - Check training status and fix issues

Run this if training failed or training_log.json is missing:


## Step 8: LoRA 重みの確認

In [ ]:
import os
from pathlib import Path

lora_dir = Path("/content/lora_weights")
print("📁 Output files:")
print()

for file in sorted(lora_dir.glob("*")):
    if file.is_file():
        size_mb = file.stat().st_size / (1024 * 1024)
        print(f"  {file.name:40s} {size_mb:>8.2f} MB")

# メインの LoRA 重みファイル
lora_model = lora_dir / "anime-impressionist-lora.safetensors"
if lora_model.exists():
    size_mb = lora_model.stat().st_size / (1024 * 1024)
    print(f"\n✅ Main LoRA weights: {lora_model.name} ({size_mb:.2f} MB)")

## Step 9: Google Drive に結果を保存

In [ ]:
# 学習済み LoRA 重みを Google Drive に保存
import shutil
from pathlib import Path

lora_weights_dir = Path("/content/lora_weights")
drive_output_dir = Path("/content/drive/MyDrive/lora_weights")

# 出力ディレクトリ作成
drive_output_dir.mkdir(parents=True, exist_ok=True)

# ファイルをコピー
for file in lora_weights_dir.glob("*"):
    if file.is_file():
        destination = drive_output_dir / file.name
        shutil.copy2(file, destination)
        print(f"✅ Saved: {file.name}")

print(f"\n📁 Output saved to: {drive_output_dir}")
print("\n💾 Download instructions:")
print("1. Open Google Drive")
print("2. Navigate to /lora_weights/")
print("3. Download 'anime-impressionist-lora.safetensors'")
print("4. Place in: anime-character-generator/ root directory")

## Step 10: 推論テスト (オプション)

トレーニング済み LoRA モデルで推論テストを実行

In [ ]:
# LoRA を使用した推論テスト
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image

print("⏳ Loading model...")

# パイプライン作成
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
)
pipe = pipe.to("cuda")

# LoRA 重みロード
try:
    # PEFT を使用した LoRA ロード
    from peft import PeftModel
    # (LoRA ロード実装)
    print("✅ LoRA weights loaded")
except:
    print("⚠️ LoRA loading requires additional setup")

# テストプロンプト
prompt = "1girl, watercolor painting style, masterpiece, high quality, anime"
negative_prompt = "low quality, nsfw"

print(f"\n🎨 Generating: {prompt}...")

# 画像生成
with torch.no_grad():
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        height=512,
        width=512,
        num_inference_steps=20,
        guidance_scale=7.0
    ).images[0]

# 結果表示
image.save("/content/test_output.png")
print("✅ Image saved: test_output.png")
display(image)

## ✅ 完了！

**次のステップ:**
1. LoRA 重みをダウンロード
2. ローカルで `character_generator.py --use_lora=True` で推論
3. 結果を確認

**参考:**
- [GitHub リポジトリ](https://github.com/Shion1124/anime-character-generator)
- [HuggingFace モデル](https://huggingface.co/runwayml/stable-diffusion-v1-5)

## Step 11: HuggingFace Hub へのアップロード

トレーニング済み LoRA モデルを HuggingFace Model Hub で公開


In [ ]:
# Step 11: HuggingFace Hub へのアップロード

from huggingface_hub import HfApi, create_repo
import json
from pathlib import Path

# 認証（事前に huggingface-cli login を実行）
api = HfApi()

# リポジトリ設定
username = "YOUR_USERNAME"  # 自分の HuggingFace ユーザー名に変更
repo_name = "anime-impressionist-lora"
repo_id = f"{username}/{repo_name}"

print(f"🚀 HuggingFace Hub にアップロード開始: {repo_id}")

# テンプレートREADME.md作成
readme_content = """# 🎨 anime-impressionist-lora

**文体:** 印象派風、水彩画風、アニメキャラクター  
**モデル:** Stable Diffusion v1.5 + LoRA (Low-Rank Adaptation)  
**ファイルサイズ:** 約 4 MB (軽量・高速)

## 📊 モデル情報

| 項目 | 詳細 |
|------|------|
| **ベースモデル** | `runwayml/stable-diffusion-v1-5` |
| **ファインチューニング手法** | LoRA (PEFT) |
| **トレーニングデータ** | Danbooru 300 枚 (5 スタイル) |
| **学習時間** | 約 1-2 時間 (Colab T4 GPU) |
| **LoRA ランク** | 8 |
| **LoRA アルファ** | 32 |

## 🚀 使用方法

### 方法 1: diffusers + PEFT (推奨)

```python
import torch
from diffusers import StableDiffusionPipeline
from peft import PeftModel

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
)

pipe.unet = PeftModel.from_pretrained(
    pipe.unet,
    "{repo_id}",
    adapter_name="anime_lora"
)

pipe = pipe.to("cuda")

prompt = "1girl, watercolor painting style, masterpiece, anime"
image = pipe(prompt=prompt, num_inference_steps=20).images[0]
image.save("output.png")
```

## 📈 トレーニング詳細

- **データセット**: Danbooru 300 枚 (5 スタイル)
- **ハイパーパラメータ**: lr=1e-4, rank=8, alpha=32
- **エポック**: 50
- **最適化**: AdamW

## 📋 ライセンス

[OpenRAIL License](https://huggingface.co/spaces/CompVis/stable-diffusion-license)

## 🤝 参考

- [anime-character-generator](https://github.com/Shion1124/anime-character-generator)
- [PEFT Documentation](https://github.com/huggingface/peft)
"""

# README.md を一時保存
readme_path = Path(lora_weights_dir) / "README.md"
with open(readme_path, "w", encoding="utf-8") as f:
    f.write(readme_content)

print(f"✅ README.md を生成しました ({len(readme_content)} 文字)")

# リポジトリ作成（既存の場合はスキップ）
try:
    api.create_repo(repo_id=repo_id, repo_type="model", exist_ok=True, private=False)
    print(f"✅ リポジトリを作成: {repo_id}")
except Exception as e:
    print(f"⚠️ リポジトリ作成: {e}")

# アップロード対象ファイル
files_to_upload = [
    ("anime-impressionist-lora.safetensors", "LoRA 重みファイル"),
    ("adapter_config.json", "LoRA 設定ファイル"),
    ("training_log.json", "トレーニングログ"),
    ("README.md", "モデルカード")
]

print("\n📤 ファイルをアップロード中...")
for filename, description in files_to_upload:
    file_path = Path(lora_weights_dir) / filename
    
    if not file_path.exists():
        print(f"⚠️ {filename} が見つかりません (スキップ)")
        continue
    
    try:
        api.upload_file(
            path_or_fileobj=str(file_path),
            path_in_repo=filename,
            repo_id=repo_id,
            repo_type="model"
        )
        file_size = file_path.stat().st_size / (1024 * 1024)  # MB
        print(f"✅ {filename} ({file_size:.1f} MB) - {description}")
    except Exception as e:
        print(f"❌ {filename}: {e}")

print("\n" + "="*60)
print(f"🎉 アップロード完了！")
print(f"📍 モデル URL: https://huggingface.co/{repo_id}")
print(f"💡 使用方法: model = {repo_id}")
print("="*60)